In [43]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')
seed = 123


### Passo a passo do processo
- Dividir o dataset em amostras menores com ou sem repetição de dados;
- Treinar o modelo de cada amostra preferencialmente com abordagens distintas;
- Obter predições usando cada um dos modelos treinados;
- Calcular a média da previsão de cada modelo ou, em caso de respostas binárias, escolher os modelos que obtiveram os mesmos resultados com maior frequência.


### Implementando o Bagging

In [44]:
df  =  pd.read_csv('diabetes_.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [45]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [46]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [47]:
df.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [48]:
268/500

0.536

In [49]:
X = df.drop("Outcome",axis="columns")
y = df.Outcome

In [50]:
scaler = StandardScaler()
X_scal = scaler.fit_transform(X)
X_scal[:3]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scal,y, stratify=y,random_state=seed)

print(f'Base treino: {X_train.shape}')
print(f'Base teste: {X_test.shape}')

Base treino: (576, 8)
Base teste: (192, 8)


In [52]:
y_test.value_counts()

0    125
1     67
Name: Outcome, dtype: int64

In [53]:
scores = cross_val_score(DecisionTreeClassifier(),X,y,cv=5)
scores

array([0.65584416, 0.66233766, 0.68831169, 0.78431373, 0.71895425])

In [54]:
scores.mean()

0.7019522960699431

In [55]:
modelo_bagg = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=seed
)
modelo_bagg.fit(X_train,y_train)
scores = cross_val_score(modelo_bagg,X,y, cv=5)
print(f'Acurácia do modelo Bagging = {round(modelo_bagg.oob_score_*100,2)}%')
print(f'Acurácia do modelo padrão = {round(modelo_bagg.score(X_test,y_test)*100,2)}%')
print(f'Acurácia do modelo validação cruzada = {round(scores.mean()*100,2)}%')

Acurácia do modelo Bagging = 73.61%
Acurácia do modelo padrão = 78.12%
Acurácia do modelo validação cruzada = 75.92%
